# ProcessTransects

Similar to ProcessGrid, this notebook extracts nodestrings from the model grid 2D mesh for use in plotting transects. Each given nodestring will be converted to an ESRI shapefile of points and their corresponding depths.

## Parameters

 * `ssm_full_grid_file`: The 2DM file that defines the model grid
 * `nodestrings`: A dict keyed by nodestring number (indexed starting from 1, just like Aquaveo SMS), with values as a filename labeling string for the output shapefile
 * `output_pattern`: A format string to use to generate shapefile paths and filenames. The one parameter is the values of the `nodestrings` dict.
 * `crs`: The Coordinate Reference System (map projection) that the 2DM model grid uses (UTM zone 10)

In [1]:
ssm_full_grid_file = "SSM_Grid/Salish_Sea_Shelf_top_0.2_DZ_10_3_16.2dm"

nodestrings = {
    57: "sjf_to_georgia",
    58: "sjf_thru_puget_sound"
}
output_pattern = "gis/transect_{0}.shp"

crs = "epsg:32610"

import py2dm
from shapely.geometry import Point, Polygon
import geopandas as gpd
import numpy as np

In [2]:
node_ids = []
node_points = []
node_depths = []
with py2dm.Reader(ssm_full_grid_file) as mesh:
    for node in mesh.iter_nodes():
        node_ids.append(node.id)
        node_points.append(Point(node.pos[:2]))
        node_depths.append(node.pos[2])
        
    node_points_df = gpd.GeoDataFrame({
        "depth": node_depths,
        "geometry": node_points
    }, index=node_ids, crs=crs)
    node_points_df.index.name='node_id'

    for i,node_string in enumerate(mesh.node_strings):
        # SMS uses nodestring numbers starting from 1, so this convention is preserved in
        # designating notestrings to extract
        nsnum = i + 1
        if nsnum in nodestrings:
            # Pandas DataFrames must be indexed by a list. Keeping node_string.nodes
            # as a tuple will cause Pandas to think we're looking up a MultiIndex
            nodestring_df = node_points_df.loc[list(node_string.nodes)]
            display(nodestring_df)
            nodestring_df.to_file(output_pattern.format(nodestrings[nsnum]))

,depth,geometry
node_id,,
1159,257.800720,POINT (374011.682 5370035.740)
1252,191.153000,POINT (377292.015 5370100.970)
1339,191.626007,POINT (380243.734 5368997.670)
1338,191.858002,POINT (383314.237 5367358.850)
1423,185.763000,POINT (386738.325 5366096.140)
...,...,...
1438,132.710999,POINT (163580.169 5658128.690)
1353,128.296005,POINT (163017.595 5660179.920)
1267,115.012001,POINT (162024.906 5662431.840)


,depth,geometry
node_id,,
1159,257.800720,POINT (374011.682 5370035.740)
1252,191.153000,POINT (377292.015 5370100.970)
1339,191.626007,POINT (380243.734 5368997.670)
1338,191.858002,POINT (383314.237 5367358.850)
1423,185.763000,POINT (386738.325 5366096.140)
...,...,...
15231,35.279999,POINT (508894.550 5222601.700)
15251,41.653000,POINT (508444.620 5222082.400)
15273,48.144001,POINT (507898.250 5221873.000)
